# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

In [6]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('/Users/veronikaplonsak/scaling_to_production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [12]:
from sklearn.model_selection import train_test_split, cross_validate

X = adult_dt.drop(columns = 'income')
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state= 42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

In [ ]:
# What is the random state of the splitting function?
#Answer:
#T he random state is a parameter of the splitting function that initializes the random number generator.
# It controls the randomness during data splitting and ensures reproducibility of results.

# Why is it useful? 
# Answer: Setting a random state ensures that every time you run the splitting function with the same random state value, you get the same output. 
# This reproducibility is crucial for debugging, testing, and sharing code, as it guarantees consistent results in data partitioning, which is fundamental in training and evaluating machine learning models. 
# Additionally, the random state controls the shuffling applied to the data before applying the split, ensuring that the split is consistent across multiple function calls.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import cross_validate

num_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

pipe_num = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('standardizer', RobustScaler())
])

pipe_cat = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OHE', OneHotEncoder(drop='if_binary', handle_unknown='infrequent_if_exist'))
])

ctransform = ColumnTransformer([
    ('num_transform', pipe_num, num_cols),
    ('cat_transforms', pipe_cat, cat_cols)
])

ctransform


ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('standardizer',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OHE',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [36]:
from sklearn.ensemble import RandomForestClassifier
Pipeline = Pipeline([
    ('preprocessing', ctransform),
    ('classifier', RandomForestClassifier())
])
Pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('standardizer',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OHE',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [37]:
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']
res_dict= cross_validate(pipe, X_train, Y_train, cv = 5, scoring=scoring,return_train_score = True)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [38]:
res = pd.DataFrame(res_dict)
res.sort_values(by=['test_neg_log_loss'])

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
2,2.658193,0.100512,-0.389368,-0.081304,0.901110,1.0,0.853664,0.999945,0.776343,0.999964
4,2.665168,0.100100,-0.374868,-0.081859,0.901096,1.0,0.856077,1.000000,0.776705,1.000000
0,2.717124,0.101808,-0.369965,-0.081517,0.904469,1.0,0.848651,1.000000,0.773181,1.000000
1,2.668073,0.099704,-0.360330,-0.081765,0.902317,1.0,0.848212,1.000000,0.769511,1.000000
3,2.756956,0.100687,-0.354976,-0.081990,0.908315,1.0,0.859368,0.999945,0.784723,0.999887


Calculate the mean of each metric. 

In [39]:
res.mean()

fit_time                   2.693103
score_time                 0.100562
test_neg_log_loss         -0.369901
train_neg_log_loss        -0.081687
test_roc_auc               0.903461
train_roc_auc              1.000000
test_accuracy              0.853195
train_accuracy             0.999978
test_balanced_accuracy     0.776092
train_balanced_accuracy    0.999970
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [40]:
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, log_loss

pipe.fit(X_train,Y_train)

Y_pred = pipe.predict(X_test)
Y_pred_proba = pipe.predict_proba(X_test)

roc_auc = roc_auc_score(Y_test,Y_pred_proba[:,1])
accuracy = accuracy_score(Y_test,Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test,Y_pred)
neg_log_loss = -1*log_loss(Y_test,Y_pred_proba) 

print('ROC AUC score on test data is ', roc_auc)
print('Accuracy score on test data is ', accuracy)
print('Balanced accuracy score on test data is ', balanced_accuracy)
print('Negative log loss score on test data is ', neg_log_loss)

ROC AUC score on test data is  0.8999367857968903
Accuracy score on test data is  0.8540280479066434
Balanced accuracy score on test data is  0.7745744127687473
Negative log loss score on test data is  -0.38455159438564807


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

In [ ]:
# Answer:

# Since we are working with binary data, where income is categorized as either '>50K' or '<=50K', we need to encode the income into a binary format: 0 or 1. 
# Although this encoding can be done at a later stage, we prefer to ensure our target values are correctly encoded from the start to facilitate further transformations. Additionally, as we are using pipelines, incorporating this transformation into the pipeline isn't necessary. 
# By default, pipelines do not transform target values, which would require more complex code than the simple lambda function we applied during data loading. 
# This approach simplifies the code, making it easier to read and understand.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.